In [258]:
import sys
import warnings
import re

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

DATA_PATH = "../data"

In [259]:
# Load excel file
df = pd.read_excel(DATA_PATH + "/consumo_material_clean.xlsx")

## Preprocessing

In [260]:
# Separate code into two columns
new_columns = df["CODIGO"].str.extract(r"([a-zA-Z]+)([0-9]+)", expand=False)
df["CODIGO_CLASS"] = new_columns[0]
df["CODIGO_NUM"] = new_columns[1]
df.drop(columns=["CODIGO"], inplace=True)

In [261]:
# FECHAPEDIDO to datetime in day/month/year format
df["FECHAPEDIDO"] = pd.to_datetime(df["FECHAPEDIDO"], dayfirst=True)
df.sort_values(by=["FECHAPEDIDO"], inplace=True)
df.reset_index(drop=True, inplace=True)

/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_8214/38037239.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["FECHAPEDIDO"] = pd.to_datetime(df["FECHAPEDIDO"], dayfirst=True)


In [262]:
# assert all rows in origen follow number-number-number format
def fix_origen_typos(origen_string):
    numbers = re.findall(r"[0-9]+", origen_string)
    return "-".join(numbers)


df["ORIGEN"] = df["ORIGEN"].apply(fix_origen_typos)

In [263]:
# separate ORIGEN in three columns by '-'
origin_separated_columns = df["ORIGEN"].str.split("-", expand=True)
df["PURCHASING_REGION"] = origin_separated_columns[0]
df["PURCHASING_HOSPITAL"] = origin_separated_columns[1]
df["PURCHASING_DEPARTMENT"] = origin_separated_columns[2]
df.drop(columns=["ORIGEN"], inplace=True)

In [264]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [265]:
# basic date features
def generate_date_features(df):
    df["YEAR"] = df["FECHAPEDIDO"].dt.year
    df["MONTH"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.month / 12)
    df["DAYOFMONTH"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.day / 31)
    df["DAYOFYEAR"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.dayofyear / 365)
    return df

In [266]:
def add_timeseries_features(df):
    df["ROLLING_MEAN_3M"] = df["CANTIDADCOMPRA"].rolling(90).mean()
    df["WEIGHTED_MEAN_3M"] = (
        df["CANTIDADCOMPRA"]
        .rolling(90)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    df["ROLLING_MEAN_1Y"] = df["CANTIDADCOMPRA"].rolling(365).mean()
    df["WEIGHTED_MEAN_1Y"] = (
        df["CANTIDADCOMPRA"]
        .rolling(365)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    return df

In [267]:
# get pairings of product and hospital and create a dict "product number": list of hospitals
pairings = df.groupby(["CODIGO_NUM", "PURCHASING_HOSPITAL"]).size().reset_index()
pairings = pairings.groupby(["PURCHASING_HOSPITAL"])["CODIGO_NUM"].apply(list)
pairings = pairings.to_dict()

In [268]:
def generate_train_test_df(full_df):
    # Get train and test sets
    train = full_df[full_df["YEAR"] < 2023]
    X_train = train.drop(columns=["CANTIDADCOMPRA", "FECHAPEDIDO"])
    y_train = train["CANTIDADCOMPRA"]

    test = full_df[full_df["YEAR"] == 2023]
    X_test = test.drop(columns=["CANTIDADCOMPRA", "FECHAPEDIDO"])
    y_test = test["CANTIDADCOMPRA"]

    return train, X_train, y_train, test, X_test, y_test

In [269]:
def train_model_cv(X_train, y_train, X_test, y_test):
    # GroupKFold
    group_kfold = GroupKFold(n_splits=8)
    groups = X_train["YEAR"]

    model = XGBRegressor(random_state=42, n_estimators=500)

    val_losses = []
    test_losses = []

    for idx, (train_index, test_index) in enumerate(
        group_kfold.split(X_train, y_train, groups)
    ):
        X_train_group, X_val_group = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_group, y_val_group = y_train.iloc[train_index], y_train.iloc[test_index]

        model.fit(X_train_group, y_train_group)

        y_val_pred = model.predict(X_val_group)
        val_loss = mean_squared_error(y_val_group, y_val_pred, squared=False)
        val_losses.append(val_loss)

        y_test_pred = model.predict(X_test)
        test_loss = mean_squared_error(y_test, y_test_pred, squared=False)
        test_losses.append(test_loss)

    return np.mean(val_losses), np.mean(test_losses)

In [270]:
def train_model_eval(X_train, y_train, X_test, y_test):
    model = XGBRegressor(random_state=42, n_estimators=500)
    model.fit(X_train, y_train)

    y_test_pred = model.predict(X_test)
    test_loss = mean_squared_error(y_test, y_test_pred, squared=False)
    mape_error = mean_absolute_percentage_error(y_test, y_test_pred)

    return test_loss, mape_error

In [271]:
def plot_model_predictions(
    model, train, X_train, y_train, test, X_test, y_test, plot_train=False
):
    plt.figure(figsize=(30, 8))
    X_train["PREDICTION"] = model.predict(X_train)
    X_train["REAL"] = y_train
    X_train["FECHAPEDIDO"] = train["FECHAPEDIDO"]

    X_test["PREDICTION"] = model.predict(X_test)
    X_test["REAL"] = y_test
    X_test["FECHAPEDIDO"] = test["FECHAPEDIDO"]

    if plot_train:
        sns.lineplot(
            x="FECHAPEDIDO",
            y="PREDICTION",
            data=X_train,
            marker="o",
            label="train preds",
        )
        sns.lineplot(
            x="FECHAPEDIDO", y="REAL", data=X_train, marker="o", label="train real"
        )

    sns.lineplot(
        x="FECHAPEDIDO", y="PREDICTION", data=X_test, marker="o", label="test preds"
    )
    sns.lineplot(x="FECHAPEDIDO", y="REAL", data=X_test, marker="o", label="test real")

    plt.legend()
    plt.show()

In [272]:
%%time
# iterate over each hospital and list of products
columns = ["FECHAPEDIDO", "CANTIDADCOMPRA"]

product_hospital_losses = pd.DataFrame(columns=["PRODUCT", "HOSPITAL", "MSE", "MAPE"])
for hospital, product_list in pairings.items():
    for product in product_list:
        partial_df = df.loc[
            (df["PURCHASING_HOSPITAL"] == hospital) & (df["CODIGO_NUM"] == product)
        ][columns]
        partial_df = generate_date_features(partial_df)
        partial_df = add_timeseries_features(partial_df)
        if 2023 not in partial_df["YEAR"].unique() or partial_df["YEAR"].nunique() == 1:
            continue
        train, X_train, y_train, test, X_test, y_test = generate_train_test_df(
            partial_df
        )
        test_loss, mape_error = train_model_eval(X_train, y_train, X_test, y_test)

        product_hospital_losses = pd.concat(
            [
                product_hospital_losses,
                pd.DataFrame(
                    [[product, hospital, test_loss, mape_error]],
                    columns=["PRODUCT", "HOSPITAL", "MSE", "MAPE"],
                ),
            ]
        )

CPU times: user 32.4 s, sys: 52 s, total: 1min 24s
Wall time: 44.6 s


In [277]:
product_hospital_losses.sort_values(by=["MSE"], ascending=False)[:20]

,PRODUCT,HOSPITAL,MSE,MAPE
0,65159,10,2459.864258,0.351409
0,66071,7,2162.997939,0.380000
0,64751,10,1503.476732,0.888560
0,65159,4,1414.213476,0.250000
0,66071,13,1078.829477,0.497560
0,64765,7,880.820746,0.662275
0,64932,10,879.445566,1.654300
0,64764,13,872.657422,0.372783
0,65159,18,733.729702,0.095940
0,64764,7,707.106514,0.125000
